# 模型文件地址
    
    https://github.com/PaddlePaddle/models/tree/develop/PaddleCV/image_classification
    
    wget http://paddle-imagenet-models-name.bj.bcebos.com/ResNet50_pretrained.tar
    tar -xvf ResNet50_pretrained.tar
    
# 初始化环境

    pip install opencv-python
    pip install paddlepaddle==1.5
    
    

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import cv2
import time
import sys
import math
import numpy as np
import argparse
import functools

import paddle
import paddle.fluid as fluid

#绘图函数
import matplotlib
#服务器环境设置
import matplotlib.pyplot as plt


#加载自定义文件
from sdk.resnet import ResNet50
from sdk.attack_pp import *


# 创建对抗样本相关的损失函数和标签并加载模型

In [2]:
#http://www.paddlepaddle.org.cn/documentation/docs/zh/1.5/user_guides/howto/prepare_data/index_cn.html

class_dim=1000
image_shape=[3,224,224]
model_name="resnet50"
use_gpu=False
pretrained_model="ResNet50_pretrained"
TOPK=3

adv_program=fluid.Program()

#完成初始化
with fluid.program_guard(adv_program):
    input_layer = fluid.layers.data(name='image', shape=image_shape, dtype='float32')
    #设置为可以计算梯度
    input_layer.stop_gradient=False

    # model definition
    model = ResNet50()
    out = model.net(input=input_layer, class_dim=class_dim)
    out = fluid.layers.softmax(out)

    place = fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()
    exe = fluid.Executor(place)
    exe.run(fluid.default_startup_program())

    #记载模型参数
    fluid.io.load_persistables(exe, pretrained_model)
    
#设置adv_program的BN层状态
init_prog(adv_program)      
    
    
#创建测试用评估模式
eval_program = adv_program.clone(for_test=True)

#定义梯度
with fluid.program_guard(adv_program):
    label = fluid.layers.data(name="label", shape=[1] ,dtype='int64')
    loss = fluid.layers.cross_entropy(input=out, label=label)
    #http://www.paddlepaddle.org.cn/documentation/docs/zh/1.5/api_cn/backward_cn.html
    gradients = fluid.backward.gradients(targets=loss, inputs=[input_layer])[0]

  

# 预测原始图片

In [3]:
img_path = "mug227.png"

img=process_img(img_path)

fetch_list = [out.name]

print(img.shape)

result = exe.run(eval_program,
                 fetch_list=fetch_list,
                 feed={ 'image':img })
result = result[0][0]

pred_label = np.argsort(result)[::-1][:TOPK]
print("Test-score: {0}, class {1}".format(result[pred_label], pred_label))
sys.stdout.flush()


o_img=tensor2img(img)


error: OpenCV(4.1.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:3718: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


# 调用FGSM算法 无定向攻击

In [ ]:
img_path = "mug227.png"
img=process_img(img_path)

#生成对抗样本
#step_size控制了扰动的大小
adv=FGSM(adv_program=adv_program,eval_program=eval_program,gradients=gradients,o=img,
         input_layer=input_layer,output_layer=out,step_size=64.0/256.0,epsilon=16.0/256,isTarget=False,target_label=0)

    
result = exe.run(eval_program,
                 fetch_list=fetch_list,
                 feed={ 'image':adv })
result = result[0][0]

pred_label = np.argsort(result)[::-1][:TOPK]

print("Test-score: {0}, class {1}".format(result[pred_label], pred_label))
sys.stdout.flush()

adv_img=tensor2img(adv)

show_images_diff(o_img,adv_img)

# 演示不同$l_{inf}$约束下FGSM无定向攻击的效果

In [ ]:
img_path = "mug227.png"
img=process_img(img_path)

for epsilon in [8.0/256,16.0/256,32.0/256,64.0/256]:

    #生成对抗样本
    #step_size控制了扰动的大小
    adv=FGSM(adv_program=adv_program,eval_program=eval_program,gradients=gradients,o=img,
             input_layer=input_layer,output_layer=out,step_size=128.0/256.0,epsilon=epsilon,isTarget=False,target_label=0)


    result = exe.run(eval_program,
                     fetch_list=fetch_list,
                     feed={ 'image':adv })
    result = result[0][0]

    pred_label = np.argsort(result)[::-1][:TOPK]

    print("Test-score: {0}, class {1} epsilon={2}".format(result[pred_label], pred_label,epsilon))
    sys.stdout.flush()

    adv_img=tensor2img(adv)

    show_images_diff(o_img,adv_img)

# 调用FGSM算法 定向攻击

In [ ]:
img_path = "mug227.png"
img=process_img(img_path)

#生成对抗样本
adv=FGSM(adv_program=adv_program,eval_program=eval_program,gradients=gradients,o=img,
         input_layer=input_layer,output_layer=out,step_size=64.0/256.0,epsilon=64.0/256,isTarget=True,target_label=443)

    
result = exe.run(eval_program,
                 fetch_list=fetch_list,
                 feed={ 'image':adv })
result = result[0][0]

pred_label = np.argsort(result)[::-1][:TOPK]

print("Test-score: {0}, class {1}".format(result[pred_label], pred_label))
sys.stdout.flush()

adv_img=tensor2img(adv)

#展示原始图片和对抗样本的差异
show_images_diff(o_img,adv_img)

# 演示不同$l_{inf}$约束下FGSM定向攻击的效果

In [ ]:
img_path = "mug227.png"
img=process_img(img_path)

for epsilon in [8.0/256,16.0/256,32.0/256,64.0/256]:

    #生成对抗样本
    #step_size控制了扰动的大小
    adv=FGSM(adv_program=adv_program,eval_program=eval_program,gradients=gradients,o=img,
             input_layer=input_layer,output_layer=out,step_size=64.0/256.0,epsilon=epsilon,isTarget=True,target_label=443)


    result = exe.run(eval_program,
                     fetch_list=fetch_list,
                     feed={ 'image':adv })
    result = result[0][0]

    pred_label = np.argsort(result)[::-1][:TOPK]

    print("Test-score: {0}, class {1} epsilon={2}".format(result[pred_label], pred_label,epsilon))
    sys.stdout.flush()

    adv_img=tensor2img(adv)

    show_images_diff(o_img,adv_img)

# 调用PGD算法 无定向攻击

In [ ]:
img_path = "mug227.png"
img=process_img(img_path)

#生成对抗样本
#step_size控制了扰动的大小
adv=PGD(adv_program=adv_program,eval_program=eval_program,gradients=gradients,o=img,
         input_layer=input_layer,output_layer=out,step_size=2.0/256.0,epsilon=16.0/256,iteration=20,isTarget=False,target_label=0)

    
result = exe.run(eval_program,
                 fetch_list=fetch_list,
                 feed={ 'image':adv })
result = result[0][0]

pred_label = np.argsort(result)[::-1][:TOPK]

print("Test-score: {0}, class {1}".format(result[pred_label], pred_label))
sys.stdout.flush()

adv_img=tensor2img(adv)

show_images_diff(o_img,adv_img)

# 调用PGD算法 定向攻击

In [ ]:
img_path = "mug227.png"
img=process_img(img_path)

#生成对抗样本
#step_size控制了扰动的大小
adv=PGD(adv_program=adv_program,eval_program=eval_program,gradients=gradients,o=img,
         input_layer=input_layer,output_layer=out,step_size=1.0/256.0,epsilon=16.0/256,iteration=20,isTarget=True,target_label=968)

    
result = exe.run(eval_program,
                 fetch_list=fetch_list,
                 feed={ 'image':adv })
result = result[0][0]

pred_label = np.argsort(result)[::-1][:TOPK]

print("Test-score: {0}, class {1}".format(result[pred_label], pred_label))
sys.stdout.flush()

adv_img=tensor2img(adv)

show_images_diff(o_img,adv_img)